In [1]:
import pandas as pd
import xlrd
from xlrd import open_workbook
from scipy.stats import pearsonr 
from scipy.stats import powerlaw
from collections import Counter
from scipy.stats import pearsonr
from scipy.stats import linregress 
import powerlaw
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import warnings
import numpy as np
import scipy as sp
import pprint
import operator
import math
from math import sin, cos, sqrt, atan2, radians
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
excel_file ='DataRotterdam.xlsx'
linknetwork= pd.read_excel(excel_file, sheet_name='Links',index_col=0)
nodenetwork= pd.read_excel(excel_file, sheet_name='NodeCode', index_col=0)
coordinates= pd.read_excel(excel_file, sheet_name='Coordinates', index_col=0)
demandnetwork= pd.read_excel(excel_file, sheet_name='Demand', index_col=0)
lines=pd.read_excel(excel_file, sheet_name='Lines', index_col=0)
link_count = linknetwork.shape[0]
node_count = nodenetwork.shape[0]
line_count=lines.shape[0]
demand_count=demandnetwork.shape[0]
coordinates = coordinates.shape[0]
book = open_workbook('DataRotterdam.xlsx')
link_data = book.sheet_by_name('Links') #read link file and correct sheet
node_data = book.sheet_by_name('NodeCode') #read node file and correct sheet
coor_data = book.sheet_by_name('Coordinates') #read coordinate file
line_data = book.sheet_by_name('Lines')
demand_data = book.sheet_by_name('Demand')
NodeLabel={} #create dictionary to name nodes based on station label
NodeLines={} #create dictionary to determine which lines serve which node
NodeHACode={}
for i in range(link_count):
    if link_data.cell_value(i+1, 2) not in NodeLabel and link_data.cell_value(i+1, 2) != 8000:
        for j in range(node_count):
            if int(node_data.cell_value(j+1, 0)) == int(link_data.cell_value(i+1, 2)):
                NodeLabel[int(link_data.cell_value(i+1, 2))]= node_data.cell_value(j+1, 1)
                NodeHACode[int(link_data.cell_value(i+1, 2))]= node_data.cell_value(j+1, 3)  
    if link_data.cell_value(i+1, 3) not in NodeLabel and link_data.cell_value(i+1, 3) != 8000:
        for j in range(node_count):
            if int(node_data.cell_value(j+1, 0)) == int(link_data.cell_value(i+1, 3)):
                NodeLabel[int(link_data.cell_value(i+1, 3))]= node_data.cell_value(j+1, 1)
                NodeHACode[int(link_data.cell_value(i+1, 3))]= node_data.cell_value(j+1, 3) 
for i in NodeLabel:
    NodeLines[i]=[]
    for j in range(link_count):
        if link_data.cell_value(j+1,2) == i and link_data.cell_value(j+1, 2) != 8000:
            if link_data.cell_value(j+1,5) not in NodeLines[i]:
                if link_data.cell_value(j+1,5) not in ['A', 'B', 'C', 'D', 'E']:
                    NodeLines[i].append(int(link_data.cell_value(j+1,5)))
                else:
                    NodeLines[i].append(str(link_data.cell_value(j+1,5)))
        if link_data.cell_value(j+1,3) == i and link_data.cell_value(j+1, 3) != 8000:
            if link_data.cell_value(j+1,5) not in NodeLines[i]:
                if link_data.cell_value(j+1,5) not in ['A', 'B', 'C', 'D', 'E']:
                    NodeLines[i].append(int(link_data.cell_value(j+1,5)))
                else:
                    NodeLines[i].append(str(link_data.cell_value(j+1,5)))

#Compute the number of internal nodes and links where external nodes van a label value of > 9900
Internal_nodes=0
for i in NodeLabel:
    if i < 9900:
        Internal_nodes += 1
IntLinks=0
for i in range(link_count):
    if link_data.cell_value(i+1, 2) < 9900 or link_data.cell_value(i+1, 3) < 9900:
        IntLinks += 1
        
#Import demand data and create OD matrix
DemandData={}
for i in range(1, demand_count+1):
    for j in range(1, demand_count+1):
        #if (int(demand_data.cell_value(i,0) >= 9900 and int(demand_data.cell_value(0,j)) >= 9900)):
            #DemandData[int(demand_data.cell_value(i,0)),int(demand_data.cell_value(0,j))] = 0  
        #else:
        DemandData[int(demand_data.cell_value(i,0)),int(demand_data.cell_value(0,j))]= demand_data.cell_value(i,j)

#read the characteristics of each Line        
LineData={} 
for i in range(line_count):
    if line_data.cell_value(i+1,0) not in ['A', 'B', 'C', 'D', 'E']:
        LineData[int(line_data.cell_value(i+1,0))]={}
    else:
        LineData[str(line_data.cell_value(i+1,0))]={}
    LineData[line_data.cell_value(i+1,0)]["Modality"]= line_data.cell_value(i+1,1)
    LineData[line_data.cell_value(i+1,0)]["Speed"]= line_data.cell_value(i+1,2)
    LineData[line_data.cell_value(i+1,0)]["Frequency"]= line_data.cell_value(i+1,3)
        

In [3]:
#Determine the location of each node and validate each node is given a location
NodeLat={}
NodeLeng={}
NodeValidation={}
for i in NodeHACode:
    for j in range(1,coordinates+1):
        if NodeHACode[i] == coor_data.cell_value(j,1):
            NodeLat[i]= coor_data.cell_value(j,3)
            NodeLeng[i]= coor_data.cell_value(j,4)
            NodeValidation[i]= "Yes"
    #some stops are not in the coordinate data set due to temporal rerouting, these are checked manually
    if i not in (NodeValidation):
        NodeLat[i]=0 
        NodeLeng[i]=0
        NodeValidation[i]= "No"    

In [4]:
#Determine the distance between nodes which can be used to check if nodes are to closed and also to determine the link distance
R = 6373.0 # approximate radius of earth in km
InList=[]
Dist={}
for i in NodeLabel:
    Dist[i]={}
    for j in NodeLabel:
        if i != j:
            Dist[i][j]=0
            if NodeLat[i] != 0:
                lat1 = radians(float(NodeLat[i]))
                lon1 = radians(float(NodeLeng[i]))
                lat2 = radians(float(NodeLat[j]))
                lon2 = radians(float(NodeLeng[j]))

                dlon = lon2 - lon1
                dlat = lat2 - lat1

                a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
                c = 2 * atan2(sqrt(a), sqrt(1 - a))
                Dist[i][j]= R * c * 1000
                distance = R * c * 1000 #calculate distance in meters between every node
#                 if distance < 150: # if the distance is smaller than 150 metres for two nodes, print the nodes
#                     if NodeLabel[i] != NodeLabel[j]:
#                         if ([NodeLabel[i], NodeLabel[j]]) not in InList:
#                             print (NodeLabel[i], '\&' , NodeLabel[j],  Dist[i][j])
#                             InList.append([NodeLabel[i], NodeLabel[j]])
#                             InList.append([NodeLabel[j], NodeLabel[i]])

In [5]:
# #Prevent links going to the same node in case of a terminal station with the same label
# for i in range(link_count):
#     if NodeLabel[int(link_data.cell_value(i+1, 2))] == NodeLabel[int(link_data.cell_value(i+1, 3))]:
#         print(int(link_data.cell_value(i+1, 2)), NodeLabel[int(link_data.cell_value(i+1, 2))],  int(link_data.cell_value(i+1, 5)))

In [6]:
# # If there are duplicates in the NodeLabels, this code can filter them
# Duplicates=[]
# DuplicatesNo={}
# for k in (NodeLabel):
#     for l in (NodeLabel):
#         if k != l:
#             if NodeLabel[k] == NodeLabel[l]:
#                 if NodeLabel[k] not in Duplicates:
#                     Duplicates.append(NodeLabel[k])
# for i in Duplicates:
#     DuplicatesNo[i]=[]
#     for j in NodeLabel:
#         if NodeLabel[j] == i:
#             DuplicatesNo[i].append(j)
# # for i in DuplicatesNo:
# #     for j in DuplicatesNo[i]:
# #         for k in DuplicatesNo[i]:
# #              if j != k:
# #                 if Dist[j][k] > 200:
# #                     print(i, j, NodeLines[j])
# #                     print(i, k, NodeLines[k])
# DuplicatesNo

In [7]:
# for i in range(link_count):
#         if Dist[link_data.cell_value(i+1,2)][link_data.cell_value(i+1,3)] > 3000:
#             if link_data.cell_value(i+1,2) < 9900 and link_data.cell_value(i+1,3) < 9900:
#                 print(round(Dist[link_data.cell_value(i+1,2)][link_data.cell_value(i+1,3)],1), 
#                   NodeLabel[link_data.cell_value(i+1,2)], NodeLabel[link_data.cell_value(i+1,3)], link_data.cell_value(i+1,5))

In [8]:
LineIndex=[] #give PT line index number 
LineStops=[] #number of stops per line
for i in range(link_count): #loop over all nodes
    if link_data.cell_value(i+1, 5) not in LineIndex: #if a line number is not added yet
        if link_data.cell_value(i+1,5) not in ['A', 'B', 'C', 'D', 'E']:
            LineIndex.append(int(link_data.cell_value(i+1, 5))) #add line index
        else:
            LineIndex.append(link_data.cell_value(i+1, 5)) #add line index
        LineStops.append(1) #set number of stops to 1 initially
for i in range(len(LineIndex)): #loop over all links
    index=LineIndex[i] #set index to recall the line
    x=[] #create variable x to define the highest stopnumber for a line
    for j in range(link_count+1):
        if link_data.cell_value(j, 5) == index: #if a node is served by a line
            x = link_data.cell_value(j, 1) #set x to value of stopnumber
            if x > int(LineStops[i]): #if x is higher than current highest stop number
                LineStops[i]=int(x) #LineStops becomes x                                
NumberOfLinks={}
for i in range(len(LineIndex)):
    j=LineIndex[i]
    k=LineStops[i]
    NumberOfLinks[j] =  k
LineOverview={} 
StopsPerLine={}
for j in LineIndex:
    StopsPerLine[j]=[]
    LineOverview[j]=[]
    for i in range(link_count+1):
        if link_data.cell_value(i, 5) == j:
            for k in range(1,(NumberOfLinks[j]+1)):
                if link_data.cell_value(i,1) == k: 
                    if k == 1:
                        if link_data.cell_value(i, 2) != 8000:
                            StopsPerLine[j].append(NodeLabel[int(link_data.cell_value(i, 2))])
                            LineOverview[j].append(int(link_data.cell_value(i, 2)))
                    if link_data.cell_value(i, 3) != 8000:
                        StopsPerLine[j].append(NodeLabel[int(link_data.cell_value(i, 3))])
                        LineOverview[j].append(int(link_data.cell_value(i, 3)))
G=nx.empty_graph(NodeLabel,create_using=nx.Graph())
#G=nx.empty_graph(NodeNo,create_using=nx.Graph()) #create graph
for i in LineIndex: #Loop over number of Lines
    for j in range(len(LineOverview[i])):
        if j != len(LineOverview[i])-1:
            k=j+1
            x=LineOverview[i][j]
            y=LineOverview[i][k]
            G.add_edge(x,y) #Add a link between nodes k & l as it serves link j-1 - j for line index                        
#nx.draw_networkx(G, with_labels=True) #draw the graph
#nx.write_gexf(G, "Amsterdam.gexf") #export the graph to gephi file
A = nx.adjacency_matrix(G)
A.shape[0]


638

In [9]:
edge_lines={}
P_Graph = nx.Graph() #Create the P_Graph for element A and B
G=nx.Graph() #Create graph in the L-space for element C and network visualization 
for i in NodeLabel:
    G.add_node(i)
    if i < 9900: #only add internal nodes 
        P_Graph.add_node(i)
for i in LineIndex: #Loop over number of Lines
    for j,k in zip(range(len(LineOverview[i])-1), range(1,len(LineOverview[i]))):
            x=LineOverview[i][j]
            y=LineOverview[i][k]
            link_weight = Dist[x][y] / LineData[i]["Speed"]*3.6
            for add in range(1,link_count+1):
                if (link_data.cell_value(add, 5)) == i:
                       if (link_data.cell_value(add, 2)) == x:
                              if (link_data.cell_value(add, 3)) == y:
                                    if link_data.cell_value(add, 6) != '':
                                        link_weight += link_data.cell_value(add, 6)*60 #add stop time
                                        if link_data.cell_value(add, 7) != '':
                                                link_weight += link_data.cell_value(add, 7)*300 #add transfer time                
            G.add_edge(x,y, weight = link_weight) #Add a link between nodes k & l as it serves link j-1 - j for line index
    for j in LineOverview[i]:
        for k in LineOverview[i]:
            if j != k:
                if (j < 9900 and k < 9900): #Only add internal links as external nodes are excluded
                    P_Graph.add_edge(j,k)

#Determine clustering coefficient for Element B
Clustering = nx.clustering(P_Graph)

#Determine eigenvector centrality for Element A
EV = nx.eigenvector_centrality(P_Graph, max_iter=100)
Max_EV = sorted(EV.items(), key=operator.itemgetter(1), reverse=True)
EV_Weight = {}
for i in EV:
    EV_Weight[i] = EV[i] / (Max_EV[0][1])

In [10]:
#Create GG graph for shortest paths where each node is a combination of a node and line
GG=nx.DiGraph()
GG_Label={}
GG_Node={}
for i in NodeLabel:
    GG_Node= 'origin',i
    GG.add_node(GG_Node)
    GG_Node= 'destination',i
    GG.add_node(GG_Node)
for i in LineIndex:
    GG_Label[i]=[]
    for j in LineOverview[i]:
        GG_Label[i].append(j)
    for j in GG_Label[i]:
        GG_Node = i,j
        GG.add_node(GG_Node)

GG_weight=0        
for i in GG.nodes:
    k=i[1]
    for j in GG.nodes:
        if k==j[1]:
            if i[0] != j[0]:
                if (i[0] != 'origin' and i[0] != 'destination'):
                    for freq in range(1,line_count+1):
                        if line_data.cell_value(freq,0) == j[0]:
                            GG.add_edge(i,j, weight = 180 + 3600/int(line_data.cell_value(freq,3))/2)
                elif i[0] == 'origin' and j[0] != 'destination':
                     GG.add_edge(i,j, weight = 0) #give high value of wait to prevent going through origin and destination nodes
                elif i[0] == 'destination' and j[0] != 'origin':
                     GG.add_edge(j,i, weight = 0) #give high value of wait to prevent going through origin and destination nodes
    if (i[0] != 'origin' and i[0] != 'destination'):
        for l in G.edges(i[1]):
            for m in GG.nodes:
                if i[0] == m[0]:
                    if m[1] == l[1]:
                        GG_weight = G.get_edge_data(i[1], m[1])['weight'] + 60 #plus stopping time
                        GG.add_edge(m,i, weight = GG_weight)
        

In [11]:
#Determine where the common corridors are located to prevent possible transfers in corridors by removing transfer edges
CommonCorridor={}
CommonEdges={}
for i in G.edges:
    CommonEdges[i]=[]
    for j in GG.edges:
        if j[0][1] == i[0] and j[1][1] == i[1]:
            if j[0][0] not in CommonEdges[i]:
                CommonEdges[i].append(j[0][0])            
        elif j[0][1] == i[1] and j[1][1] == i[0]:
            if j[0][0] not in CommonEdges[i]:
                CommonEdges[i].append(j[0][0])
for i in CommonEdges:
    if len(CommonEdges[i]) > 1:
        CommonCorridor[i]=CommonEdges[i]
CorridorNodes=[]
for i in CommonCorridor:
    for j in CommonCorridor:
        if i != j:
            if i[0] == j[0] or i[0] == j[1]:
                if G.degree(i[0]) <= 2:
                    if CommonCorridor[i] == CommonCorridor[j]:
                        if i[0] not in CorridorNodes:
                            CorridorNodes.append(i[0])    
            elif i[1] == j[0] or i[1] == j[1]:
                        if G.degree(i[1]) <= 2:
                            if CommonCorridor[i] == CommonCorridor[j]:
                                if i[1] not in CorridorNodes:
                                    CorridorNodes.append(i[1])
CorridorEdges=[]
for i in GG.edges:
    if i[0][0] != 'origin' and i[1][0] != 'destination':
        if i[0][1] == i[1][1]:
            if i[0][1] in CorridorNodes:
                CorridorEdges.append(i)
for i in CorridorEdges:
    GG.remove_edge(i[0],i[1])

In [12]:
#this code determines the shortest paths for each node pair with demand based on the weight of the edges
SP_list=[]
pair=[]
for i in GG.nodes:
    if i[0] == 'origin': #start with origin
        for j in GG.nodes:
            if i[1] != j[1]:
                if j[0] == 'destination': #end with destination
                    if int(DemandData[(i[1], j[1])]) != 0: #only calculcate the shortest path if there is demand for the nodepair
                        pair=i,j                    
                        SP_list.append(pair)
SP={}
for i in range(len(SP_list)):
    SP[(SP_list[i][0][1], SP_list[i][1][1])]= nx.shortest_path(GG, source= SP_list[i][0], target=SP_list[i][1],weight='weight')    
    

In [13]:
#Determine the transfer share for each node as part of Element C
Transfer={}
for i in NodeLabel:
    Transfer[i]=0
for i in SP:
    O=i[0]
    D=i[1]
    Demand= int(DemandData[(i[0], i[1])])
    Path=SP[i][1:-1]
    for j,k in zip(range(1,len(Path)-1),range(2,len(Path))):
        if Path[j][0] != Path[k][0]:
            Transfer[Path[j][1]] += Demand
Transfers={}
TransferShare={}
TOT_Transfers=0
for i in NodeLabel:
    if i < 9900:
        TOT_Transfers += Transfer[i]
for i in Transfer:
    if i < 9900: #only for internal nodes
        TransferShare[i]=Transfer[i]/TOT_Transfers
        if Transfer[i] > 0:
            Transfers[i]=math.log10(Transfer[i]) #take the log of transfers
        else:
            Transfers[i]=0
Max_Trans = max(Transfers.items() , key=operator.itemgetter(1))[1]
for i in Transfers:
    Transfers[i] /= Max_Trans
sorted_Transfers = sorted(Transfers.items(), key=operator.itemgetter(1), reverse=True)

In [14]:
node={} #Create dictionary for node information
eA={}
eB={}
eC={}
H={} #Create dict for sorting the hierarchical degree of nodes
HD={}
for i in (NodeLabel):
    if i < 9900:
        if G.degree[i] > 2: #only if degree is higher than 2 the hierarchical degree can be computed
            eA[i]=EV_Weight[i]   #variable for eA
            eB[i]=(1-Clustering[i]) #formula for eB
            eC[i]=(Transfers[i]*((G.degree[i]-2)/G.degree[i])) #formula for eC
        else: #if the degree is lower or equal than 2, no values can be computed and are set to 0
            eA[i]=0
            eB[i]=0
            eC[i]=0
        H[NodeLabel[i]]=eA[i]*eB[i]*eC[i] #calculate the hierarchical degree
        HD[i]= eA[i]*eB[i]*eC[i]
        if H[NodeLabel[i]] != 0: #for every node which has a hierarchical degree which is not 0
            node[i]= {'Node ID': i,
                      'degree' : G.degree[i],
                      'eA' : eA[i],
                      'eB' : eB[i],
                      'eC' : eC[i],
                      'Hierarchical degree' : H[NodeLabel[i]]}

#Create sorted lists to determine the highest scoring nodes for each element
sorted_H = sorted(HD.items(), key=operator.itemgetter(1), reverse=True)
sorted_eA = sorted(eA.items(), key=operator.itemgetter(1), reverse=True)
sorted_eB = sorted(eB.items(), key=operator.itemgetter(1), reverse=True)
sorted_eC = sorted(eC.items(), key=operator.itemgetter(1), reverse=True)

In [15]:
for i in range(0,5):
    j= i+1
    print(j,'&', NodeLabel[sorted_eC[i][0]],'&',round(sorted_eC[i][1],3), '&', j,'&', NodeLabel[sorted_Transfers[i][0]],'&',round(sorted_Transfers[i][1],3))

1 & Station Schiedam Centrum & 0.739 & 1 & Beurs & 1.0
2 & Zuidplein & 0.714 & 2 & Station Blaak & 0.901
3 & Station Blaak & 0.676 & 3 & Station Schiedam Centrum & 0.887
4 & Beurs & 0.667 & 4 & Zuidplein & 0.844
5 & Stadhuis/Weena & 0.585 & 5 & Stadhuis/Weena & 0.819


In [16]:
for i in range(0, 10):
    get = sorted_H[i][0]
    j = i+1
    print(j, '.', NodeLabel[get], '&', round(HD[get],3), '&', round(eA[get],3), '&', round(eB[get],3), '&', round(eC[get],3), '\\')

1 . Beurs & 0.503 & 1.0 & 0.755 & 0.667 \
2 . Stadhuis/Weena & 0.446 & 0.974 & 0.783 & 0.585 \
3 . Station Schiedam Centrum & 0.412 & 0.72 & 0.775 & 0.739 \
4 . Marconiplein & 0.317 & 0.943 & 0.745 & 0.451 \
5 . Station Blaak & 0.295 & 0.643 & 0.68 & 0.676 \
6 . Eendrachtsplein & 0.201 & 0.568 & 0.722 & 0.492 \
7 . Kruisplein & 0.188 & 0.911 & 0.727 & 0.283 \
8 . Zuidplein & 0.167 & 0.264 & 0.882 & 0.714 \
9 . Station Alexander & 0.138 & 0.345 & 0.752 & 0.532 \
10 . Schiedam Nieuwland & 0.129 & 0.588 & 0.572 & 0.382 \


In [17]:
sorted_H[0][0]

8005

In [18]:
for i in DemandData:
    if i[0] == 8019:
        if DemandData[i] != 0:
            print(NodeLabel[i[1]], DemandData[i])
    elif i[1] == 8019:
        if DemandData[i] != 0:
            print(NodeLabel[i[0]], DemandData[i])

Brielselaan 0.002
Putsebocht 1.719
Breeplein 3.409
Kerstendijk 1.479
Smeetslandsedijk 0.749
Grote Hagen 0.071
Station Lombardijen 5.912
Bredenoord 0.709
Appeldijk 0.574
Herenoord 1.128
Nieuwenoord 0.064
P+R Beverwaard 0.425
Ahoy'-complex 1.586
Zuiderparkweg 0.574
Akkeroord 2.02
Croystraat 0.148
Oude Watering 1.029
Schinnenbaan 0.378
Beverwaardseweg 0.306
Korperweg 0.013
Waalhaven Zuidzijde 0.578
Kaatsbaan 0.107
Rietdijk 0.005
Motorstraat 0.013
De Twee Heuvels 0.002
Strevelsweg 0.003
Hillevliet 1.472
Arendsweg 0.791
Verboomstraat 2.728
Sportlaan(Rotterdam) 0.098
Den Hamweg 0.134
Ophemertstraat 0.061
Zaltbommelstraat 0.686
Eemhavenweg 0.356
Neptunusstraat 0.29
Rondoplein 0.174
Asterlo 3.071
Larenkamp 0.347
Zuiderbegraafplaats 0.195
Victor Hugoweg 4.513
Van de Woestijnestraat 2.502
Dumasstraat 11.077
Amelandseplein 0.064
Molenwei 4.194
Gruttostraat 0.043
Keizerswaard 0.068
Klein Nieuwland 2.684
Spoorweghaven 0.117
RDM Campus 1.497
Burgdorfferstraat 0.072
Station Zuid 0.037
Kreekhuizenlaan

KeyError: 8000

In [ ]:
#Get the results for the highest scoring nodes
Results={}
for i in range(0,20): #Reverse loop to begin at the end of the list
    Results[(sorted_H[i][0])]=[NodeLabel[sorted_H[i][0]],round(sorted_H[i][1],3),round(eA[sorted_H[i][0]],3)
                                , round(eB[sorted_H[i][0]],3), round(eC[sorted_H[i][0]],3)]
    print(Results[(sorted_H[i][0])])

In [ ]:
#Draw a map of the graph based on latitude and longtitude coordinates
sorted_ID = sorted(HD.items(), key=operator.itemgetter(1)) #order nodes from low to high
H = nx.Graph() #create graph for drawing
colormap=[] #color of nodes based on H value
size=[] #size of nodes based on H value
for i in range(len(sorted_ID)): 
    if sorted_ID[i][0] < 9900: #only internal nodes
        H.add_node(sorted_ID[i][0], pos=(float(NodeLeng[sorted_ID[i][0]]),float(NodeLat[sorted_ID[i][0]])))
        colormap.append(math.log10(1+float(sorted_ID[i][1])))
        size.append(10+1500*math.log10(1+float(sorted_ID[i][1])))    
for i in G.edges:
    if (i[0] < 9900 and i[1] < 9900): #only fully internal links
        H.add_edge(i[0],i[1])
# NodeRank={}
# rank=0
# for i in reversed(sorted_ID):
#     rank += 1
#     if rank < 10:
#         NodeRank[i[0]] = rank
#     else:
#         NodeRank[i[0]]= 
        
#Plot the figure        
plt.figure(figsize=(18,12))
# H=nx.relabel_nodes(H,NodeRank) # can be used to label the nodes
plt.title('Hierarchy of nodes in the Rotterdam PTN ', fontsize = 18, weight='bold', verticalalignment='center')
nx.draw(H, nx.get_node_attributes(H,'pos'), with_labels=False, line_edge=1, node_size=size, node_color = colormap,
        vmin = 0, vmax=0.15, cmap='viridis', linewidths=1, edgecolors='black', legend=True)
sm = plt.cm.ScalarMappable(cmap=plt.cm.get_cmap('viridis'), norm=plt.Normalize(vmin = 0, vmax=0.4))
sm._A = []
cbar = plt.colorbar(sm, orientation="horizontal", fraction=0.033, pad=0) 
cbar.set_label(label='H$_\mathbf{i}$ values', size=15, weight='bold')
cbar.ax.xaxis.set_label_position('top')
cbar.ax.xaxis.set_ticks_position('top')
plt.savefig('NodeHierarchy010NoCentral.png')

plt.show()

# nx.write_gexf(H, "Rotterdam.gexf")

In [ ]:
HD

In [ ]:
#Function for power-law distribution calculation
#for dist_type in [HD, eA, eB, eC]:
dist_type = HD
c = {}
values=np.arange(0,1,0.05)
data=[]
for j in values:
    Counting = 0
    for i in dist_type.values():
        if i >= j:
            Counting += 1
            if j > 0:
                data.append(j)
        c[round(j,2)]=Counting / len(HD)
x=[]
y=[]
for i in c:
    if i > 0 and c[i] > 0:
        x.append(np.log(i))
        y.append(np.log(c[i]))
slope, intercept, r_value, p_value, std_err = sp.stats.linregress(x, y)
k=slope

#Create the plot
plt.figure(figsize=(8,6))
plt.style.use('bmh')
ax = plt.axes(yscale='log')
alpha_label = '\u03B1 ~ {}' .format(str(round(k,3)))
plt.ylim(0.001, 1.1)
plt.xlim(-0.02,1.0)
plt.title('Distribution function of H$\mathdefault{^i}$')
plt.xlabel('H$\mathdefault{^i}$ values', fontsize=14)
plt.ylabel('Fraction of nodes with a higher H$\mathdefault{^i}$ value', fontsize=14)
plt.scatter(*zip(*sorted(c.items())), marker='o', s=60, linewidths=1,  edgecolors='k',zorder=2)
plt.text(0.42, 0.15, alpha_label, color='r', fontsize=14,
        bbox={'facecolor': 'white', 'linestyle': 'solid',
         'linewidth': 2, 'edgecolor': 'black', 'alpha': 0.2, 'pad': 6})


rangedata = np.linspace(0.001, 1.0,100000)        
plt.plot(rangedata,np.power(rangedata,k)/(1.7*len(HD)),'r',zorder=1);
plt.savefig('PowerDistribution010.png')
plt.show();



In [ ]:
Counting=0
for i in NodeLabel:
    if i < 9900:
        if HD[i] != 0:
            Counting += 1
print(Counting)

In [ ]:
Counting

In [ ]:
from scipy.stats import norm
for x,y,z in zip([eA, eB, eC, Transfers],['eA', 'eB', 'eC','Transfers'],
                 ['e$\mathdefault{^A}$', 'e$\mathdefault{^B}$', 'e$\mathdefault{^C}$', 'Normalized transfer share']):
    data=[]
    for i in x:
        data.append(x[i])
    data_sorted= list(sorted(data))
    p = 1. * np.arange(len(data)) / (len(data) - 1)

    plt.figure(figsize=(6,4))
    plt.style.use('bmh')
    plt.plot(data_sorted, p)
    plt.ylim(0.70,1.01) 
    plt.xlim(-0.02,1.01)
    plt.ylabel('Cumulative distribution')
    if z != 'Normalized transfer share':
        plt.xlabel('{} values' .format(z))
    else:
        plt.xlabel('{}' .format(z))
    if z != 'Normalized transfer share':
        plt.title('Cumulative distribution function of {}'.format(z), fontsize=12 )
    else:
        plt.title('Cumulative distribution function of transfer share', fontsize=12 )
    plt.savefig('CDF{}010.png' .format(y))
    plt.show()

In [ ]:
def gini(list_of_values): #Algorithm to define the gini index
    sorted_list = sorted(list_of_values)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2.
    return (fair_area - area) / fair_area

Internal=[] #Add variable list for all nodes with a hierarchical degree of not zero
for i in HD:
    if i < 9900:
        Internal.append(HD[i])
Cummulative=0
Entry=list(range(1,len(Internal)+1)) #ranking for sorted hierarchical degrees based on number of entries
Ranking=sorted(Internal) #the ranking of the hierarchical degree list
Height=[0]
for i in Internal:
    Cummulative += i
for i in Ranking:
    Height.append(Height[-1]+i)
Height.pop(0) #remove first value which was added as initial
gini = round(gini(Internal),3) #the gini index value with 2 decimals

#Create the plot
plt.figure(figsize=(8,6))
plt.style.use('bmh')
x = Entry
Equal=[]
for i in Entry:
    Equal.append(i/len(Entry)*Cummulative)
y1 = Equal
y2 = Height
plt.plot([0,len(Internal)],[0,Cummulative])
plt.plot(Entry,Height)
plt.xlim(0,len(Internal))
plt.ylim(0,Cummulative)
plt.text(120, 4.5, 'Gini index = {}'.format(gini), color='b', fontsize=16,
        bbox={'facecolor': 'white', 'linestyle': 'solid',
         'linewidth': 2, 'edgecolor': 'black', 'alpha': 0.2, 'pad': 6})
plt.tick_params(labelsize=12)
plt.xticks(np.arange(0, len(Internal)+1,len(Internal)/5), ('0', '0.2', '0.4', '0.6', '0.8','1'))
plt.grid(False)
plt.text(520, 0.7, 'Lorenz curve', rotation=75, fontsize=14, weight='bold', zorder=2)
plt.text(225, 2.2, 'Line of Equality', rotation=37, fontsize=14, weight='bold', zorder=2)
plt.ylabel('Cummulative hierarchical degree of nodes', fontsize=12 )
plt.xlabel('Fraction of nodes', fontsize=12)
plt.fill_between(x, y1, y2, facecolor='red', interpolate=True, alpha=0.5, zorder=1)
plt.fill_between(x, 0, y2, facecolor='green', interpolate=True, alpha=0.5)
plt.title("Gini Index of the hierarchical degree ");
plt.savefig('GiniDistribution010.png')


In [ ]:
# #Verification of the transfer model
# Verification=[]
# VerificationSP=[]
# for i in SP:
#     VerificationSP.append(i) 
# import random
# for i in range (0,10):
#     j=random.randint(0,len(VerificationSP))
#     Verification.append(VerificationSP[j])
# for i in Verification:
#     print(SP[i])    

In [ ]:
#Statistical analysis of nodes
Between= nx.betweenness_centrality(G) #use networkX tool to create dict with betweenness for each node
Statistics={} #Dict to capture all statistics of a node
As={} #Dict for assortativity
Modes={} #Dict for the modes serving each node for overlapping degree
OverDeg={} #Dict for degree in each layer for each node
for i in HD:
    if HD[i] != 0:
        Statistics[i]={}
        Statistics[i]['H']=HD[i]
        Statistics[i]['k']=G.degree[i]
        Statistics[i]['eA']=eA[i]
        Statistics[i]['eB']=eB[i]
        Statistics[i]['eC']=eC[i]
        As[i]=0 
        for j in G.edges(i):
            As[i] += G.degree(j[1])
        Statistics[i]['A']=As[i]/len(G.edges(i)) #Average degree of neighbors
        Statistics[i]['C']=Clustering[i]
        Modes[i]={} # add dict for each node to create a list for each mode
        OverDeg[i]=[] #add list to determine degree for each mode
        for j in NodeLines[i]:
            if LineData[j]['Modality'] not in Modes[i]: #the mode did not exist yet for the node
                Modes[i][LineData[j]['Modality']]=[j]
            else:
                Modes[i][LineData[j]['Modality']].append(j) #if the mode is already in the list
        for j in Modes[i]:
            for k in Modes[i][j]:
                for l in range(len(LineOverview[k])): #find the location of the node on the line
                    if LineOverview[k][l] == i:
                        if l != 0:
                            if [j, LineOverview[k][l-1]] not in OverDeg[i]: #add one node before node i to the list
                                OverDeg[i].append([j, LineOverview[k][l-1]])
                        if l != len(LineOverview[k])-1:
                            if [j, LineOverview[k][l+1]] not in OverDeg[i]: #add one node fater node i to the list
                                OverDeg[i].append([j, LineOverview[k][l+1]])
        Statistics[i]['O']=len(OverDeg[i])
        Statistics[i]['B']=Between[i]    

In [ ]:
pearsonr(Stat['H'], Stat['A'])


In [ ]:
Stat={}   
for j in ['eA','eB','eC','H','k','A','C','O','B']:
    Stat[j]=[]
    for i in Statistics:    
        Stat[j].append(Statistics[i][j])

#Plot the pearson correlation between H and the elements in one figure

fig, (axs) = plt.subplots(2, 3, figsize=(15,10))
fig.suptitle('Correlation between elements and hierarchical degree', fontsize=18)
plt.style.use('bmh')
(ax1, ax2, ax3), (ax4, ax5, ax6) = axs
for i,j, k, xlabels, ylabels in zip([ax1, ax2, ax3, ax4, ax5, ax6],['eA','eB','eC', 'eA','eA','eB'],['H','H','H', 'eB','eC','eC'],
                                    ['e$\mathdefault{^A}$','e$\mathdefault{^B}$','e$\mathdefault{^C}$', 
                                     'e$\mathdefault{^A}$','e$\mathdefault{^A}$','e$\mathdefault{^B}$'],
                                    ['H','H','H', 'e$\mathdefault{^B}$','e$\mathdefault{^C}$','e$\mathdefault{^C}$']):
    i.grid(b=None)
    if k == 'H':
        i.set_ylim([0, 0.8])
        height=0.59
    else:
        i.set_ylim([0, 1.2])
        height=0.88
    corr, _ = pearsonr(Stat[j], Stat[k])
    slope, intercept, r_value, p_value, std_err = linregress(Stat[j], Stat[k])
    x=np.linspace(0, 1.0,100000)
    i.plot(x, intercept + slope*x, 'r-')
    i.scatter(Stat[j], Stat[k], s=30, edgecolor='black', linewidth='0.75')
    i.set(xlabel='{} values'.format(xlabels), ylabel='{} values'.format(ylabels))
    SlopeLabel = str(round(slope,3)) 
    i.text(0.02, height, 'Pearson correlation: %s \n\nSlope regression line: %s \n\nStandard error: %s ' 
           % (round(corr,3), round(slope,2) , round(std_err,2)) , 
           color='k', fontsize=11, bbox={'facecolor': 'white', 'linestyle': 'solid', 'linewidth': 2, 
                                         'edgecolor': 'black', 'alpha': 0.8, 'pad': 6})   
plt.savefig('Correlation.png')
plt.plot;

In [ ]:
#Total travel time in the network
Tot_Dem=0
Tot_TT=0
for j in SP:
    for i in range(len(SP[j])-1):
        Tot_TT += (GG.get_edge_data((SP[j][i]),(SP[j][i+1]))['weight']) * DemandData[j]
    Tot_Dem += DemandData[j]
print('Total daily travel time =', int(Tot_TT/3600), 'hours')
print('Total daily demand =', int(Tot_Dem), 'passengers')
print('Average daily travel time =', int(Tot_TT/Tot_Dem/60), 'minutes')

In [ ]:
# #Create a new excel file to couple nodes from different datasets
# import xlsxwriter

# # Create a new Excel file and add a worksheet.
# workbook = xlsxwriter.Workbook('Dataset.xlsx')
# worksheet = workbook.add_worksheet()
# worksheet.write(0,0,"NodeID")
# worksheet.write(0,1,"NodeLabel")
# j=1
# for i in NodeLabel:
#     worksheet.write(j, 0, i)
#     worksheet.write(j, 1, NodeLabel[i])
#     j += 1
# workbook.close()
